# Recommend Products using SparkML

This notebook is written in **Python** so the default cell type is Python. However, you can use different languages by using the `%LANGUAGE` syntax. Python, Scala, SQL, and R are all supported.

As for the notebooks, to run these you will need to register for a free Databricks
[Community Edition account](https://community.cloud.databricks.com/)

## import modules

In [3]:
from pyspark.sql import SparkSession
from pyspark import SparkContext
import pyspark.sql.functions as F

from pyspark.mllib.recommendation import ALS, MatrixFactorizationModel, Rating

## Design Schema

In [5]:
rate_schema = "`userid` string, `accoid` string, `rating` INT"
accos_schema = "`id` string, `title` string, `location` string, `price` int, `rooms` int, `rating` float, `type` string"
recommend_schema = "`userid` string, `accoid` srting, `prediction` float"

## Read csv files to Spark DataFrame

In [7]:
rating_file_location = '/FileStore/tables/rating.csv'
dfRates = spark.read.csv(rating_file_location, rate_schema)

print(dfRates.count())
display(dfRates.take(10))

userid,accoid,rating
10,1,1
18,1,2
13,1,1
7,2,2
4,2,2
13,2,3
19,2,2
12,2,1
11,2,1
1,2,2


In [8]:
print(dfRates.count())

1187

In [9]:
accos_file_location = "/FileStore/tables/accommodation.csv"
dfAccos = spark.read.csv(accos_file_location, accos_schema)

print(dfAccos.count())
display(dfAccos.take(10))

id,title,location,price,rooms,rating,type
1,Comfy Quiet Chalet,Vancouver,50,3,3.0999999046325684,cottage
2,Cozy Calm Hut,London,65,2,4.099999904632568,cottage
3,Agreable Calm Place,London,65,4,4.800000190734863,house
4,Colossal Quiet Chateau,Paris,3400,16,2.700000047683716,castle
5,Homy Quiet Shack,Paris,50,1,1.100000023841858,cottage
6,Pleasant Quiet Place,Dublin,35,5,4.300000190734863,house
7,Vast Peaceful Fortress,Seattle,3200,24,1.899999976158142,castle
8,Giant Quiet Fortress,San Francisco,3400,12,4.099999904632568,castle
9,Giant Peaceful Palace,London,1500,20,3.5,castle
10,Sizable Calm Country House,Auckland,650,9,4.900000095367432,mansion


## Aggregations

In [11]:
df_agg = dfRates.agg(F.count('userid').alias('num_ratings'),
            F.countDistinct('userid').alias('distinct_users_rating'),
            F.max('rating').alias('best_rating'),
            F.min('rating').alias('worst_rating'),
            F.avg('rating').alias('avg_rating')
           )
display(df_agg)

num_ratings,distinct_users_rating,best_rating,worst_rating,avg_rating
1187,25,5,1,2.4667228306655433


## Left Join

In [13]:
df_leftjoined = dfAccos.withColumnRenamed('rating', 'ratings').join(dfRates, dfAccos.id == dfRates.accoid, how = 'left')
print(df_leftjoined.count())
display(df_leftjoined.take(10))

id,title,location,price,rooms,ratings,type,userid,accoid,rating
1,Comfy Quiet Chalet,Vancouver,50,3,3.0999999046325684,cottage,13,1,1
1,Comfy Quiet Chalet,Vancouver,50,3,3.0999999046325684,cottage,18,1,2
1,Comfy Quiet Chalet,Vancouver,50,3,3.0999999046325684,cottage,10,1,1
2,Cozy Calm Hut,London,65,2,4.099999904632568,cottage,3,2,1
2,Cozy Calm Hut,London,65,2,4.099999904632568,cottage,2,2,4
2,Cozy Calm Hut,London,65,2,4.099999904632568,cottage,20,2,2
2,Cozy Calm Hut,London,65,2,4.099999904632568,cottage,1,2,2
2,Cozy Calm Hut,London,65,2,4.099999904632568,cottage,11,2,1
2,Cozy Calm Hut,London,65,2,4.099999904632568,cottage,12,2,1
2,Cozy Calm Hut,London,65,2,4.099999904632568,cottage,19,2,2


In [14]:
display(df_leftjoined.select('*').where(df_leftjoined.id == '100'))

id,title,location,price,rooms,ratings,type,userid,accoid,rating
100,Villa Staden,Stockholm,80000,4,4.2,house,null,null,null


In [15]:
display(df_leftjoined.select('*').where(df_leftjoined.id == '101'))

id,title,location,price,rooms,ratings,userid,accoid,rating


## Right Join

In [17]:
df_rightjoined = dfAccos.withColumnRenamed('rating', 'ratings').join(dfRates, dfAccos.id == dfRates.accoid, how = 'right')
print(df_rightjoined.count())
display(df_rightjoined.take(10))

id,title,location,price,rooms,ratings,type,userid,accoid,rating
1,Comfy Quiet Chalet,Vancouver,50,3,3.0999999046325684,cottage,10,1,1
1,Comfy Quiet Chalet,Vancouver,50,3,3.0999999046325684,cottage,18,1,2
1,Comfy Quiet Chalet,Vancouver,50,3,3.0999999046325684,cottage,13,1,1
2,Cozy Calm Hut,London,65,2,4.099999904632568,cottage,7,2,2
2,Cozy Calm Hut,London,65,2,4.099999904632568,cottage,4,2,2
2,Cozy Calm Hut,London,65,2,4.099999904632568,cottage,13,2,3
2,Cozy Calm Hut,London,65,2,4.099999904632568,cottage,19,2,2
2,Cozy Calm Hut,London,65,2,4.099999904632568,cottage,12,2,1
2,Cozy Calm Hut,London,65,2,4.099999904632568,cottage,11,2,1
2,Cozy Calm Hut,London,65,2,4.099999904632568,cottage,1,2,2


In [18]:
display(df_rightjoined.select('*').where(df_rightjoined.id == '100'))

id,title,location,price,rooms,ratings,userid,accoid,rating


In [19]:
display(df_rightjoined.select('*').where(df_rightjoined.accoid == '101'))

id,title,location,price,rooms,ratings,type,userid,accoid,rating
null,null,null,null,null,null,null,23,101,5


## Inner Join

In [21]:
df_innerjoined = dfAccos.withColumnRenamed('rating' , 'ratings').join(dfRates, dfAccos.id == dfRates.accoid, how = 'inner')
print(df_innerjoined.count())
display(df_innerjoined.take(10))

id,title,location,price,rooms,ratings,type,userid,accoid,rating
1,Comfy Quiet Chalet,Vancouver,50,3,3.0999999046325684,cottage,10,1,1
1,Comfy Quiet Chalet,Vancouver,50,3,3.0999999046325684,cottage,18,1,2
1,Comfy Quiet Chalet,Vancouver,50,3,3.0999999046325684,cottage,13,1,1
2,Cozy Calm Hut,London,65,2,4.099999904632568,cottage,7,2,2
2,Cozy Calm Hut,London,65,2,4.099999904632568,cottage,4,2,2
2,Cozy Calm Hut,London,65,2,4.099999904632568,cottage,13,2,3
2,Cozy Calm Hut,London,65,2,4.099999904632568,cottage,19,2,2
2,Cozy Calm Hut,London,65,2,4.099999904632568,cottage,12,2,1
2,Cozy Calm Hut,London,65,2,4.099999904632568,cottage,11,2,1
2,Cozy Calm Hut,London,65,2,4.099999904632568,cottage,1,2,2


In [22]:
display(df_innerjoined.select('*').where(df_innerjoined.id == '100'))

id,title,location,price,rooms,ratings,userid,accoid,rating


In [23]:
display(df_innerjoined.select('*').where(df_innerjoined.accoid == '101'))

id,title,location,price,rooms,ratings,userid,accoid,rating


## Train the model and recommend products with the model

### Train the model

In [26]:
model = ALS.train(dfRates.rdd, 20, 20)
print(type(model))

<class 'pyspark.mllib.recommendation.MatrixFactorizationModel'>

### Save the model

In [28]:
model.save(sc,'/FileStore/model_recommendations_v3.ml' )

### Use the trained model to predict what accommodations each user might be interested

In [30]:
allPredictions = None
for USER_ID in range(0, 100):
  dfUserRatings = dfRates.filter(dfRates.userid == USER_ID).rdd.map(lambda r: r.accoid).collect()
  rddPotential  = dfAccos.rdd.filter(lambda x: x[0] not in dfUserRatings)
  pairsPotential = rddPotential.map(lambda x: (USER_ID, x[0]))
  predictions = model.predictAll(pairsPotential).map(lambda p: (str(p[0]), str(p[1]), float(p[2])))
  predictions = predictions.takeOrdered(5, key=lambda x: -x[2]) # top 5
  print("predicted for user={0}".format(USER_ID))
  if (allPredictions == None):
    allPredictions = predictions
  else:
    allPredictions.extend(predictions)

predicted for user=0
predicted for user=1
predicted for user=2
predicted for user=3
predicted for user=4
predicted for user=5
predicted for user=6
predicted for user=7
predicted for user=8
predicted for user=9
predicted for user=10
predicted for user=11
predicted for user=12
predicted for user=13
predicted for user=14
predicted for user=15
predicted for user=16
predicted for user=17
predicted for user=18
predicted for user=19
predicted for user=20
predicted for user=21
predicted for user=22
predicted for user=23
predicted for user=24
predicted for user=25
predicted for user=26
predicted for user=27
predicted for user=28
predicted for user=29
predicted for user=30
predicted for user=31
predicted for user=32
predicted for user=33
predicted for user=34
predicted for user=35
predicted for user=36
predicted for user=37
predicted for user=38
predicted for user=39
predicted for user=40
predicted for user=41
predicted for user=42
predicted for user=43
predicted for user=44
predicted for user=45
predicted for user=46
predicted for user=47
predicted for user=48
predicted for user=49
predicted for user=50
predicted for user=51
predicted for user=52
predicted for user=53
predicted for user=54
predicted for user=55
predicted for user=56
predicted for user=57
predicted for user=58
predicted for user=59
predicted for user=60
predicted for user=61
predicted for user=62
predicted for user=63
predicted for user=64
predicted for user=65
predicted for user=66
predicted for user=67
predicted for user=68
predicted for user=69
predicted for user=70
predicted for user=71
predicted for user=72
predicted for user=73
predicted for user=74
predicted for user=75
predicted for user=76
predicted for user=77
predicted for user=78
predicted for user=79
predicted for user=80
predicted for user=81
predicted for user=82
predicted for user=83
predicted for user=84
predicted for user=85
predicted for user=86
predicted for user=87
predicted for user=88
predicted for user=89
predicted for user=90
predicted for user=91
predicted for user=92
predicted for user=93
predicted for user=94
predicted for user=95
predicted for user=96
predicted for user=97
predicted for user=98
predicted for user=99

### 5 Recommendations for every user

In [32]:
allPredictions

Out[84]: [('0', '76', 3.6340362053029684),
 ('0', '66', 3.5387555809515),
 ('0', '75', 3.5161224476647996),
 ('0', '49', 3.4598124917225226),
 ('0', '39', 3.452309519311184),
 ('1', '85', 2.7949320875219517),
 ('1', '18', 2.231116724409522),
 ('1', '43', 1.993245523585799),
 ('1', '6', 1.8393500634974291),
 ('1', '38', 1.8062653885433058),
 ('2', '87', 3.037426249747395),
 ('2', '66', 2.8121284046957324),
 ('2', '99', 2.717424401785232),
 ('2', '61', 2.7094455033485656),
 ('2', '76', 2.647786794426747),
 ('3', '3', 3.6285734659934294),
 ('3', '6', 3.5910693440597496),
 ('3', '30', 3.5791745708508644),
 ('3', '76', 3.4912414057016234),
 ('3', '61', 3.4328959646971855),
 ('4', '7', 1.9952494326183172),
 ('4', '18', 1.7944661223097693),
 ('4', '80', 1.7826532422196646),
 ('4', '47', 1.7449161138147713),
 ('4', '81', 1.6762453840420042),
 ('5', '6', 2.515461956037524),
 ('5', '94', 2.5000516632336334),
 ('5', '12', 2.47872604574734),
 ('5', '64', 2.330160094295521),
 ('5', '21', 2.3248098538541297),
 ('6', '6', 4.785125182270917),
 ('6', '30', 4.496509738566286),
 ('6', '12', 4.356319440492027),
 ('6', '38', 3.9049615421961312),
 ('6', '28', 3.6314363031590147),
 ('7', '54', 2.0221011358572887),
 ('7', '34', 1.9935689928216216),
 ('7', '88', 1.8164276255370437),
 ('7', '19', 1.790765243625831),
 ('7', '58', 1.7543177642201238),
 ('8', '67', 2.3581754973244466),
 ('8', '44', 2.135581467793508),
 ('8', '47', 1.915649247790011),
 ('8', '81', 1.9149263571887105),
 ('8', '24', 1.7865358515628742),
 ('9', '30', 3.4485672606183586),
 ('9', '16', 2.952070496316716),
 ('9', '6', 2.886809091230724),
 ('9', '22', 2.8680284020675084),
 ('9', '3', 2.856697815589752),
 ('10', '41', 1.557513021864533),
 ('10', '31', 1.5217864118887516),
 ('10', '74', 1.5191292231831937),
 ('10', '46', 1.463981723574951),
 ('10', '35', 1.4013996611501864),
 ('11', '51', 2.491053752590423),
 ('11', '25', 2.170106026591073),
 ('11', '87', 2.0886046462200736),
 ('11', '97', 2.0505016108039933),
 ('11', '88', 1.9849276439076593),
 ('12', '66', 2.6284211658028176),
 ('12', '99', 2.558087513797929),
 ('12', '38', 2.5472307688338858),
 ('12', '59', 2.3918844388911205),
 ('12', '61', 2.3812157413831043),
 ('13', '12', 3.800752017532952),
 ('13', '6', 3.6208248580007174),
 ('13', '30', 3.528708449355113),
 ('13', '33', 3.0931182737958127),
 ('13', '3', 3.066013939827143),
 ('14', '12', 2.4541998865415646),
 ('14', '33', 2.393778954988692),
 ('14', '6', 2.227978331386967),
 ('14', '45', 2.1663436404450125),
 ('14', '85', 2.0142967252121475),
 ('15', '85', 3.1337336381018135),
 ('15', '45', 3.1101991713160224),
 ('15', '87', 2.493948030314011),
 ('15', '55', 2.3967744571637954),
 ('15', '38', 2.1244507102247776),
 ('16', '87', 1.3080376748633178),
 ('16', '80', 1.2867582292729587),
 ('16', '85', 1.2224913881786206),
 ('16', '45', 1.200895362157964),
 ('16', '94', 1.1844055022864062),
 ('17', '87', 3.630035018011761),
 ('17', '95', 2.3327387684467293),
 ('17', '46', 2.21854182260809),
 ('17', '25', 2.1082696727592465),
 ('17', '57', 1.7288572558801938),
 ('18', '12', 3.4428830301976565),
 ('18', '6', 3.0368058281495145),
 ('18', '30', 2.726202544769632),
 ('18', '33', 2.6649870182822917),
 ('18', '16', 2.634721632314236),
 ('19', '45', 2.909863163780835),
 ('19', '30', 2.602893248675395),
 ('19', '38', 2.421506455254253),
 ('19', '59', 2.391170826125024),
 ('19', '6', 2.2580890521475614),
 ('20', '30', 3.225992361953149),
 ('20', '12', 3.1847692697227123),
 ('20', '6', 3.008822198579313),
 ('20', '16', 2.8769687529605052),
 ('20', '3', 2.7958217832865775),
 ('21', '81', 2.5050058909313453),
 ('21', '80', 2.2219997950873465),
 ('21', '32', 2.134207544506373),
 ('21', '40', 2.018659221535444),
 ('21', '69', 1.8797524294160672),
 ('22', '30', 4.732131602959785),
 ('22', '6', 4.131563679847362),
 ('22', '59', 3.93204277469847),
 ('22', '3', 3.842087539641848),
 ('22', '38', 3.8189211952646067),
 ('23', '85', 1.731468599866839),
 ('23', '87', 1.6099087591768106),
 ('23', '47', 1.

### Prediction for user '1'

In [34]:
dfUserRatings = dfRates.filter(dfRates.userid == '1').rdd.map(lambda r: r.accoid).collect()
rddPotential  = dfAccos.rdd.filter(lambda x: x[0] not in dfUserRatings)
pairsPotential = rddPotential.map(lambda x: ('1', x[0]))

In [35]:
pairsPotential.collect()

Out[86]: [('1', '1'),
 ('1', '3'),
 ('1', '6'),
 ('1', '9'),
 ('1', '10'),
 ('1', '12'),
 ('1', '13'),
 ('1', '15'),
 ('1', '16'),
 ('1', '17'),
 ('1', '18'),
 ('1', '20'),
 ('1', '21'),
 ('1', '22'),
 ('1', '26'),
 ('1', '28'),
 ('1', '29'),
 ('1', '30'),
 ('1', '31'),
 ('1', '32'),
 ('1', '33'),
 ('1', '35'),
 ('1', '38'),
 ('1', '39'),
 ('1', '41'),
 ('1', '43'),
 ('1', '46'),
 ('1', '47'),
 ('1', '49'),
 ('1', '50'),
 ('1', '57'),
 ('1', '59'),
 ('1', '61'),
 ('1', '63'),
 ('1', '64'),
 ('1', '65'),
 ('1', '66'),
 ('1', '69'),
 ('1', '70'),
 ('1', '72'),
 ('1', '74'),
 ('1', '75'),
 ('1', '76'),
 ('1', '78'),
 ('1', '79'),
 ('1', '83'),
 ('1', '84'),
 ('1', '85'),
 ('1', '86'),
 ('1', '90'),
 ('1', '93'),
 ('1', '96'),
 ('1', '99'),
 ('1', '100')]

In [36]:
predictions = model.predictAll(pairsPotential).map(lambda p: (str(p[0]), str(p[1]), float(p[2])))

In [37]:
predictions_top5 = predictions.takeOrdered(5, key=lambda x: -x[2]) # top 5

In [38]:
predictions_top5

Out[89]: [('1', '85', 2.7949320875219517),
 ('1', '18', 2.231116724409522),
 ('1', '43', 1.993245523585799),
 ('1', '6', 1.8393500634974291),
 ('1', '38', 1.8062653885433058)]

### The top five accommodations are recommended to user '1'. Note that the quality of the recommendations are not great because the dataset was so small (the predicted ratings are not very high). Still, this lab illustrates the process you'd go through to create product recommendations.

In [40]:
predictions.sortBy(lambda kv: kv[2], False).collect()

Out[90]: [('1', '85', 2.7949320875219517),
 ('1', '18', 2.231116724409522),
 ('1', '43', 1.993245523585799),
 ('1', '6', 1.8393500634974291),
 ('1', '38', 1.8062653885433058),
 ('1', '12', 1.6723034107514418),
 ('1', '28', 1.6561542140960714),
 ('1', '46', 1.6220086815450072),
 ('1', '64', 1.5411066738158334),
 ('1', '47', 1.4973364040012482),
 ('1', '30', 1.3678106320513348),
 ('1', '70', 1.3664628251217854),
 ('1', '16', 1.3543505308977895),
 ('1', '99', 1.2891278018898698),
 ('1', '49', 1.2620984553753583),
 ('1', '69', 1.2225561134526892),
 ('1', '13', 1.2130914526600858),
 ('1', '33', 1.1646518147620402),
 ('1', '90', 1.1534574264380968),
 ('1', '66', 1.118989081546843),
 ('1', '22', 1.11798672430211),
 ('1', '26', 1.081214001488163),
 ('1', '72', 1.0273407496621518),
 ('1', '76', 1.0266452402637518),
 ('1', '59', 1.0021434129956082),
 ('1', '75', 0.9911979025061737),
 ('1', '39', 0.9833186590447318),
 ('1', '29', 0.9760750067188533),
 ('1', '3', 0.9437223807852821),
 ('1', '57', 0.9309918051594841),
 ('1', '78', 0.8929897185235611),
 ('1', '96', 0.8029132251390622),
 ('1', '61', 0.7208965991718204),
 ('1', '32', 0.717230982089273),
 ('1', '20', 0.7164491259821166),
 ('1', '21', 0.6744341330798718),
 ('1', '83', 0.6644210122991838),
 ('1', '79', 0.6454606395114663),
 ('1', '17', 0.6369435968903776),
 ('1', '74', 0.5918813163260851),
 ('1', '10', 0.5746215638535745),
 ('1', '84', 0.5418548639001817),
 ('1', '63', 0.5397739331025815),
 ('1', '15', 0.5220766342409884),
 ('1', '50', 0.49221789948060657),
 ('1', '65', 0.38337180828831074),
 ('1', '35', 0.31716271031744336),
 ('1', '86', 0.3163711397995971),
 ('1', '41', 0.27859925389922413),
 ('1', '31', 0.17353488105329995),
 ('1', '93', 0.12931925215660967),
 ('1', '1', 0.12351872964010857),
 ('1', '9', -0.06703028984793569)]

### Create DataFram as dfRecommendation

In [42]:
dfRecommendation = spark.createDataFrame(allPredictions, ['userid', 'accoid', 'prediction'] )
display(dfRecommendation.take(10))

userid,accoid,prediction
0,76,3.6340362053029684
0,66,3.5387555809515
0,75,3.5161224476647996
0,49,3.4598124917225226
0,39,3.452309519311184
1,85,2.7949320875219517
1,18,2.231116724409522
1,43,1.993245523585799
1,6,1.8393500634974291
1,38,1.8062653885433058


### Find the recommendations for a specified user

#### dfRecommendation.accoid and dfAccos.id are duplicated in the joined DataFrame, could drop one of the columns

In [45]:
df_joined_rec_acco = dfRecommendation.join(dfAccos, dfRecommendation.accoid == dfAccos.id, how = 'inner').drop(dfAccos.id)
display(df_joined_rec_acco.take(10))

userid,accoid,prediction,title,location,price,rooms,rating,type
0,76,3.6340362053029684,Pleasant Calm Villa,Berlin,30,2,2.4000000953674316,house
0,66,3.5387555809515,Beautiful Private Villa,London,80,2,2.4000000953674316,house
0,75,3.5161224476647996,Large Private Place,Berlin,50,4,3.5999999046325684,house
0,49,3.4598124917225226,Big Private Villa,NYC,90,2,4.800000190734863,house
0,39,3.452309519311184,Beautiful Calm Villa,Vancouver,50,3,3.5,house
1,85,2.7949320875219517,Nice Private Shack,Auckland,55,1,4.900000095367432,cottage
1,18,2.231116724409522,Big Peaceful Hut,Melbourne,60,2,2.4000000953674316,cottage
1,43,1.993245523585799,Nice Private Hut,Melbourne,60,3,2.799999952316284,cottage
1,6,1.8393500634974291,Pleasant Quiet Place,Dublin,35,5,4.300000190734863,house
1,38,1.8062653885433058,Big Private House,San Francisco,70,4,2.9000000953674316,house


In [46]:
display(df_joined_rec_acco.select('*').where(df_joined_rec_acco.userid == '10'))

userid,accoid,prediction,title,location,price,rooms,rating,type
10,41,1.557513021864533,Big Calm Manor,Seattle,800,11,2.7,mansion
10,31,1.5217864118887516,Colossal Private Castle,Buenos Aires,1400,15,3.3,castle
10,74,1.5191292231831937,Giant Calm Fort,Melbourne,2400,12,2.3,castle
10,46,1.463981723574951,Colossal Private Castle,San Francisco,1900,15,3.7,castle
10,35,1.4013996611501864,Colossal Quiet Chateau,NYC,2300,14,4.6,castle
